In [1]:
import pandas as pd
import numpy as np
import warnings
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

# func

In [252]:
def preprocess_tp_data(path):
  tp_data = pd.read_csv(path)
  tp_data.drop_duplicates(inplace = True)
  tp_data.last_r.loc[tp_data.last_r < 0] = -1
  tp_data.state_close_estimate.fillna(tp_data.last_r , inplace = True)
  tp_data.state_max_r.fillna(tp_data.last_r , inplace = True)
  tp_data.state_MA_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_RSI_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_MA_r.loc[tp_data.state_MA_r < -1] = -1
  tp_data.state_max_r.loc[tp_data.state_max_r < -1] = -1
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:str(x).strip('][').split(', '))
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:[float(element) for element in x])
  tp_data['state_day_start'] = tp_data['state_day_start'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  tp_data['state_day_end'] = tp_data['state_day_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

  spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
  tp_data['spread'] = tp_data.state_ticker.apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
  tp_data['deviation'] = tp_data.state_ticker.apply(lambda x: spread_df.deviation.loc[spread_df.ticker == x].iloc[0])
  tp_data['point'] = tp_data.state_ticker.apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
  tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
  tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]
  return tp_data
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput120.csv')

In [267]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

#concat_chain_data['estimate_with_thresh'] = concat_chain_data.state_chain.apply(lambda x : get_return_with_max_estimate(x) )

# TP data

In [188]:
tp_data = pd.read_csv('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput270.csv')
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-05-16 13:50:00,2022-05-18 13:50:00,90.985,90.510,NaN,7.400000,3.610526,3.884211,3.610526,"[-0.13684210526312074, -0.26315789473680745, -..."
1,MRK,2,2022-05-18 18:50:00,2022-05-19 19:00:00,92.204,92.394,NaN,5.915789,NaN,NaN,-1.000000,"[0.2842105263158036, 0.6526315789474258, 1.178..."
2,MRK,1,2022-05-25 16:10:00,2022-05-26 14:20:00,93.138,92.818,NaN,2.225000,0.131250,NaN,-1.000000,"[0.44374999999999665, 0.4750000000000133, 0.13..."
3,MRK,2,2022-05-26 14:35:00,2022-06-01 19:35:00,93.028,93.458,NaN,6.902326,3.600000,5.506977,3.600000,"[-0.05116279069765351, -0.5860465116278868, -0..."
4,MRK,2,2022-06-01 19:30:00,2022-06-03 13:40:00,91.385,92.145,NaN,3.480263,-0.125000,NaN,1.388158,"[-0.006578947368433847, -0.1250000000000187, 0..."


In [371]:
data1 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
data2 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread2.csv')
data1['spread2'] = data1.ticker.apply(lambda x: data2.spread.loc[data2.ticker == x].iloc[0])
data1.loc[data1.spread != data1.spread2]

,Unnamed: 0,ticker,point,spread,deviation,spread2
0,0,AAPL,0.01,6,5,9
5,5,AMD,0.01,10,5,25
8,8,AMZN,0.01,4,5,6
12,12,BAC,0.01,8,5,9
13,13,BA,0.01,24,5,46
24,24,DE30,0.10,39,5,36
28,28,FB,0.01,8,5,41
38,38,JP225,0.10,90,5,89
50,50,MSFT,0.01,10,5,21
52,52,NFLX,0.01,24,5,81


In [259]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].describe()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r
count,380.000000,380.000000,380.000000,380.000000,380.000000
mean,0.123399,2.594735,-0.024286,0.055182,-0.042620
std,3.929826,6.020758,3.659371,3.279577,3.668570
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-0.061935,-1.000000,-1.000000,-1.000000
50%,-1.000000,0.466667,-1.000000,-1.000000,-1.000000
75%,-1.000000,2.452899,-1.000000,-1.000000,-1.000000
max,32.342268,47.935616,32.342268,33.528571,32.342268


In [260]:
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].sum()

state_close_estimate     46.891751
state_max_r             985.999242
state_MA_r               -9.228510
state_RSI_r              20.969008
last_r                  -16.195593
dtype: float64

In [266]:
tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
tp_data.estimate_with_thresh.sum()

-63.334356909509566

In [264]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."


# concat data

In [331]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/*.csv"):
    tp_data = preprocess_tp_data(path)
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)


In [332]:
concat_data['estimate_with_thresh'] = concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 15 , 0.6) )
concat_data['estimate_with_thresh2'] = concat_data.state_chain_new.apply(lambda x : x.max() * 0.6 )
concat_data.estimate_with_thresh2.loc[concat_data.estimate_with_thresh2 < 3] = -1
concat_data.estimate_with_thresh2.sum()

-1045.9376714159102

In [333]:
concat_data.estimate_with_thresh2.loc[(concat_data.estimate_with_thresh2 > 5) & (concat_data.last_r > 0)]

55       8.093388
57       5.875338
64       6.446167
72       6.993708
84       5.155794
          ...    
3740     7.032414
3743     7.045301
3746     5.560714
3794    15.051148
3849     6.582128
Name: estimate_with_thresh2, Length: 179, dtype: float64

In [334]:
a = concat_data[['state_chain_new',	'estimate_with_thresh', 'state_close_estimate' , 'estimate_with_thresh2']].loc[(concat_data.state_close_estimate - concat_data.estimate_with_thresh > 10)].iloc[1]
a

state_chain_new          [-0.16153846153846166, 0.10769230769227132, 0....
estimate_with_thresh                                             15.915385
state_close_estimate                                             29.446154
estimate_with_thresh2                                            11.510769
Name: 314, dtype: object

In [306]:
concat_data[['state_chain_new',	'estimate_with_thresh', 'state_close_estimate' , 'estimate_with_thresh2']].loc[(concat_data.state_close_estimate - concat_data.estimate_with_thresh > 0) ].iloc[:20]

,state_chain_new,estimate_with_thresh,state_close_estimate,estimate_with_thresh2


In [325]:
concat_data[['state_chain_new',	'estimate_with_thresh', 'state_close_estimate']].loc[concat_data.estimate_with_thresh < 0].sum()

estimate_with_thresh   -2671.000000
state_close_estimate   -2508.608265
dtype: float64

In [273]:
concat_data.loc[concat_data.state_close_estimate > 10].iloc[1].state_chain_new

array([ 0.06190476, -0.34285714, -0.22380952, 10.75238095, 12.3       ,
       13.49047619, 13.3       , 11.44285714, 11.87142857, 12.41904762,
       12.94285714, 11.60952381, 12.82380952, 13.15714286, 15.3952381 ,
       15.10952381, 15.27619048, 15.53809524, 16.08571429, 17.01428571,
       17.37142857, 18.2047619 , 17.41904762, 18.65714286, 17.27619048,
       15.84761905, 16.58571429, 16.3       , 17.22857143, 18.13333333,
       17.7047619 , 17.01428571, 17.60952381, 17.82380952, 18.06190476,
       18.3       , 17.53809524, 17.63333333, 18.53809524, 19.65714286,
       19.15714286, 19.7047619 , 19.25238095, 19.37142857, 19.2047619 ,
       19.06190476, 18.87142857, 17.8952381 , 18.49047619, 18.3952381 ,
       17.7047619 , 17.84761905, 17.7047619 , 17.72857143, 17.60952381,
       18.58571429, 18.56190476, 17.96666667, 17.87142857, 17.58571429,
       18.03809524, 18.3       , 18.25238095, 19.84761905, 20.82380952,
       20.3952381 , 21.3       , 21.60952381, 21.46666667, 22.34

In [335]:
total_state = len(concat_data)
profit_state = len(concat_data.loc[concat_data.state_close_estimate > 0]) 
loss_state = total_state -  profit_state

total_profit = concat_data.state_close_estimate.sum()
total_win_profit = concat_data.state_close_estimate.loc[concat_data.state_close_estimate > 0].sum()
total_loss_profit = concat_data.state_close_estimate.loc[concat_data.state_close_estimate < 0].sum()
print(total_state , loss_state , profit_state , profit_state/total_state)
print(total_profit,total_loss_profit,total_win_profit)

3864 3273 591 0.1529503105590062
170.9926023164689 -3271.0 3441.992602316469


In [35]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data.state_close_estimate.loc[concat_chain_data.state_close_estimate>0].sum()

649.9320075821444

In [ ]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data['len_state_chain']  = concat_chain_data.state_chain.apply(lambda x : len(x))
concat_chain_data = concat_chain_data.loc[concat_chain_data.len_state_chain > 2]
concat_chain_data.head(50)

In [264]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data['len_state_chain']  = concat_chain_data.state_chain.apply(lambda x : len(x))
concat_chain_data.loc[(concat_chain_data.len_state_chain <= 2) & (concat_chain_data.state_close_estimate > 0)].state_close_estimate.sum() 

4.446544523588392

In [242]:
concat_chain_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,len_state_chain,estimate_with_thresh
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[-0.11929824561405766, -0.4701754385964981, -0...",4,-1.000000
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[0.2969696969696954, 0.11515151515150576, -0.7...",4,-1.000000
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[0.09999999999989342, 8.016666666667222, 7.100...",77,17.266667
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[0.5090909090909526, 0.5090909090909526, 0.327...",9,-1.000000
7,7,MRK,2,1.663696e+09,1.663702e+09,85.906,86.016,-1.000,2.236364,-1.0,-1.000000,-1.0,"[0.32727272727274137, 1.3272727272727414, 2.14...",24,-1.000000


In [258]:
concat_data['estimate_with_thresh'] = concat_data.state_chain.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
concat_data.estimate_with_thresh.sum()

600.1157970268418

In [336]:
concat_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new,estimate_with_thresh,estimate_with_thresh2
0,0,MRK,1,2022-10-10 19:35:00,2022-10-11 13:40:00,90.712,90.482,-1.0000,0.382609,-1.0000,-1.0000,-1.0000,"[0.38260869565216427, -0.6173913043478976, -0....",7,5,0.01,1.521739,"[0.25142857142856656, -0.4057142857143351, -0....",-1.000000,-1.0
1,1,MRK,2,2022-10-11 19:05:00,2022-10-11 19:20:00,91.072,91.192,0.1000,1.100000,0.1000,0.1000,0.1000,"[0.9333333333333728, 1.1, -0.3166666666666272,...",7,5,0.01,2.000000,"[0.4666666666666675, 0.5499999999999778, -0.15...",0.050000,-1.0
2,2,MRK,1,2022-10-14 14:00:00,2022-10-14 14:40:00,92.362,92.082,-1.0000,1.278571,-1.0000,-1.0000,-1.0000,"[1.278571428571438, 0.6000000000000203, -0.114...",7,5,0.01,1.428571,"[0.8950000000000077, 0.4200000000000147, -0.07...",-1.000000,-1.0
3,3,MRK,1,2022-10-18 13:40:00,2022-10-18 13:45:00,94.194,94.034,0.6625,0.662500,0.6625,0.6625,0.6625,"[-0.7124999999999789, 0.6624999999999212]",7,5,0.01,1.750000,"[-0.4071428571428414, 0.37857142857138004]",0.378571,-1.0
4,4,MRK,2,2022-10-19 19:00:00,2022-10-21 13:40:00,93.268,93.448,-1.0000,4.377778,-1.0000,-1.0000,-1.0000,"[0.48888888888883625, 0.37777777777775146, 0.2...",7,5,0.01,1.666667,"[0.2933333333332969, 0.22666666666664714, 0.15...",-1.000000,-1.0


# Stock_tp_data

In [358]:
def get_stock_data(tp_data):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 15 , 0.6) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max())
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

In [341]:
stock_tp_data = get_stock_data(concat_data)
stock_tp_data.loc[stock_tp_data.estimate_with_thresh > -10]

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,win_count,loss_count
state_ticker,,,,,,,,
ADBE,220.964351,399.441394,194.219489,57.508718,192.366481,164.964692,11,59
ADP,21.634607,140.581104,18.673973,3.979513,20.474964,10.361655,10,41
AMGN,74.239042,222.126263,57.962405,54.166130,59.126230,41.952133,14,37
AMZN,-0.786500,36.306383,-4.574038,-10.035577,-4.574038,0.486857,2,15
AVGO,32.730735,178.463465,22.532741,29.337972,18.998962,23.484522,9,42
BMY,55.115282,214.796879,21.106899,34.847930,38.852895,9.708262,17,46
CMCSA,13.174111,133.275173,14.324923,14.689089,12.234111,-9.088179,8,42
EBAY,9.565152,24.586612,9.565152,6.619697,9.565152,4.427110,2,4
EQIX,-5.527622,6.489369,-6.661809,-5.527622,-5.527622,-6.186580,2,8


In [362]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput180.csv'
tp_data = preprocess_tp_data(path)
stock_tp_data = get_stock_data(tp_data)
stock_tp_data.loc[stock_tp_data.estimate_with_thresh > 0]

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
state_ticker,,,,,,,,,
AAPL,-4.709986,58.150310,3.864214,2.939569,2.939569,3.948284,7.372414,2,15
ADBE,13.333746,30.674538,13.333746,4.045820,13.333746,11.313499,16.123416,1,5
ATVI,6.576010,23.128999,6.451010,6.576010,6.576010,3.019888,1.682353,3,3
GILD,2.767484,26.992295,3.280882,10.418137,2.767484,0.331886,-0.252055,2,6
GOOGL,16.356431,32.070483,8.954717,4.369811,8.954717,6.291800,16.929947,2,4
MMM,19.621213,58.485941,32.039877,27.139683,31.914877,20.106938,29.729754,4,5
MRK,7.955808,36.739905,24.939808,0.564423,24.179808,10.628286,12.592000,3,5
PM,9.663136,65.003892,22.888136,8.901977,22.888136,9.220017,16.466571,3,9
TMO,7.906252,32.733842,7.568752,5.756396,7.906252,6.639116,19.834632,4,3


In [107]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0..."


In [249]:
#tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')
def get_return_with_max_estimate(state_return_list, min_thresh = 12, cut_thresh = 0.66):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

tp_data['new_estimate'] = tp_data.state_chain.apply(lambda x: get_return_with_max_estimate(x, 12, 0.66))
tp_data.new_estimate.sum()

86.98738404481048

In [152]:
tp_data.state_close_estimate.sum()

39.52315681719646

In [214]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')


In [212]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')

In [224]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
tp_data['spread'] = tp_data.state_ticker.apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
tp_data['deviation'] = tp_data.state_ticker.apply(lambda x: spread_df.deviation.loc[spread_df.ticker == x].iloc[0])
tp_data['point'] = tp_data.state_ticker.apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [244]:
x = tp_data.state_chain.iloc[0][0]

tp_data.loc[(tp_data.state_chain[0] == 1)]

KeyError: 'False: boolean label can not be used without a boolean index'

In [247]:
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [250]:
tp_data.state_close_estimate.sum()

46.89175083298029

In [248]:
tp_data

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NFLX,2,2022-11-29 15:30:00,2022-11-30 11:15:00,281.168,283.248,-1.000000,2.354808,-1.000000,-1.000000,-1.000000,"[0.14326923076923273, 0.3596153846153828, 0.29...",24,5,0.01,1.139423,"[0.12573839662447417, 0.31561181434598967, 0.2..."
376,NFLX,1,2022-11-30 14:35:00,2022-11-30 14:40:00,283.605,281.780,-1.000000,-0.523288,-1.000000,-1.000000,-1.000000,"[-0.5232876712329024, -0.9178082191780942]",24,5,0.01,1.158904,"[-0.4515366430260266, -0.791962174940908]"
377,NFLX,1,2022-12-01 20:15:00,2022-12-01 20:20:00,318.462,317.632,-1.000000,-0.592771,-1.000000,-1.000000,-1.000000,"[-0.677108433734967, -0.5927710843373147]",24,5,0.01,1.349398,"[-0.501785714285732, -0.43928571428568647]"
378,NFLX,1,2022-12-02 20:25:00,2022-12-02 20:35:00,320.746,320.056,-1.000000,0.223188,-1.000000,-1.000000,-1.000000,"[0.22318840579701454, -0.09565217391308861, -0...",24,5,0.01,1.420290,"[0.1571428571427958, -0.06734693877554192, -0...."


In [157]:
path =  '/Users/tranthong/Desktop/stock-app/stock_data_M5/'
def get_mt5_raw_data_range(ticker, start, end):
    stock_dir = path + ticker +'_2022.csv'
    rates =  pd.read_csv(stock_dir, index_col =[0])
    rates['index'] = rates['index'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    a = rates.loc[ (rates['index'] > start) & (rates['index'] < end )]
    a.reset_index(inplace = True ,drop = True)
    a['day_num'] = a.index
    return a

start = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_start.iloc[0]
end = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_end.iloc[0]
ticker = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_ticker.iloc[0]
open = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_open.iloc[0]
stoploss = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_stop_loss.iloc[0]
r = abs(open -stoploss)
data =  get_mt5_raw_data_range(ticker, start, end)

In [158]:
(data.Close - open) / r

0       0.292308
1       0.523077
2      -0.015385
3       0.446154
4       0.984615
         ...    
297    18.215385
298    18.215385
299    18.215385
300    18.061538
301    17.138462
Name: Close, Length: 302, dtype: float64